In [1]:
from bs4 import BeautifulSoup

In [67]:
import numpy as np
data_list = ['00','01','02','03','04','05','06','07','08','09',\
             '10','11','12','13','14','15','16','17','18','19','20','21']

def deal_sgml(data_list):
    con_labels = np.zeros((1,5))
    for k in data_list:
        f = open('F:\\wen\\yan_2_1\\compare\\reuters\\reut2-0'+k+'.sgm','r')
        soup = BeautifulSoup(f,'lxml')
        if k != '21':
            sample = 1000
        else:
            sample = 578
        labels = np.zeros((sample,5))
        label_list = ['topics','places','people','orgs','exchanges']
        j = 0
        for lists in label_list:
            i = 0
            for label in soup.find_all(lists):
                if label.d != None:
                    labels[i][j] = 1
                i += 1
            j += 1
        print k
        con_labels = np.r_[con_labels,labels]
    con_labels = np.delete(con_labels,0,axis=0)
    return con_labels

In [207]:
#all_label = deal_sgml(data_list)

In [76]:
true_label = np.where(all_label<1,-1,1)
np.savetxt('F:\\wen\\yan_2_1\\compare\\reuters\\data\\label.csv',true_label,delimiter=',')

In [194]:
#clear the title and cont to txt
title_data = []
cont_data = []
def deal_sgml_text(data_list):
    for k in data_list:
        f = open('F:\\wen\\yan_2_1\\compare\\reuters\\reut2-0'+k+'.sgm','r')
        soup = BeautifulSoup(f,'lxml')
        '''
        #deal with the title to text
        for text in soup.find_all('text'):
            if text.title != None:
                title_data.append(text.title.string)
            else:
                title_data.append('None')
                        
        '''
        #deal with the cont to text
        [s.extract() for s in soup(['title','dateline','author'])] #delete some title
        for text in soup.find_all('text'):
            if text.type == 'brief':
                print 'hehe'
                cont_data.append('None')
            elif text != None:
                cont_data.append(str(text))
                #print str(text)
            else:
                cont_data.append('None')
        print k
    #return cont_data



In [208]:
#deal_sgml_text(data_list)

In [193]:
j = 0
with open('F:\\wen\\yan_2_1\\compare\\reuters\\data\\cont.txt','wb') as f:
    for i in cont_data:
        i = i.replace('\n', '')
        f.write(i)
        f.write('\n')

In [192]:
len(cont_data)
#print cont_data[1]

21578

In [219]:

#clear the cont data
f0 = open('F:\\wen\\yan_2_1\\compare\\reuters\\data\\cont.txt','r')
cont = f0.readlines()
f0.close()
with open('F:\\wen\\yan_2_1\\compare\\reuters\\data\\cont1.txt','wb') as f:
    for line in cont:
        if '***Blah' in line:
            f.write('None')
            f.write('\n')
        else:
            line = line.replace('<text>','')
            line = line.replace('Reuter</text>','')
            line = line.replace('REUTER</text>','')
            f.write(line)
            #f.write('\n')


In [226]:
#filter the none of title and cont
import pandas as pd
path = "F:\\wen\\yan_2_1\\compare\\reuters\\data\\"
label = pd.read_csv(path+'label.csv',names=['topics','places','people','orgs','exchanges'])
title = pd.read_table(path+'title.txt',sep='\n',names=['title'])
cont = pd.read_table(path+'cont.txt',sep='\n',names=['cont'])
data = pd.concat([label,title,cont],axis=1)

In [233]:
for index, row in data.iterrows():
    if row['title']=='None' or row['cont']=='None':
        data = data.drop([index])
        

In [240]:
#data[['topics','places','people','orgs','exchanges']].to_csv(path+'nNone_label.csv',header=False,index=False)
with open(path+'nNone_title.txt','wb') as f:
    for line in data['title']:
        f.write(line)
        f.write('\n')
        
with open(path+'nNone_cont.txt','wb') as f:
    for line in data['cont']:
        f.write(line)
        f.write('\n')

In [24]:
#using gensim to computer tfidf
from gensim import corpora,models
from collections import defaultdict
path = "F:\\wen\\yan_2_1\\compare\\reuters\\data\\"
f = open(path+'nNone_cont.txt','r')
read_title = f.readlines()
f.close()
title =[]
for line in read_title:
    title.append(line.strip('\n'))
'''    
# 去掉停用词
stoplist = set('FOR A OF THE AND TO IN'.split())
texts = [[word for word in title.split() if word not in stoplist]
         for document in title]
         
stoplist=set('for a of the from is an and to in'.split())
texts = []
for corp in corpora_documents:
    word_list = ''
    for word in corp.lower().split():
        if word not in stoplist:
            word_list = word+' '+word_list
    texts.append(word_list)
    
# 去掉只出现一次的单词
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]


dictionary = corpora.Dictionary(title)   # 生成词典

# 将文档存入字典，字典有很多功能，比如
# diction.token2id 存放的是单词-id key-value对
# diction.dfs 存放的是单词的出现频率
dictionary.save(path+'deerwester.dict')  # store the dictionary, for future reference
'''

# 生成字典和向量语料
corpus = [x.split() for x in title]
dct = corpora.Dictionary(corpus)
corpus_as_bow = [dct.doc2bow(x) for x in corpus]
model_trained = models.TfidfModel(corpus_as_bow)
corpus_tfidf_trained = model_trained[corpus_as_bow]


In [26]:
'''
for item in corpus_tfidf_trained:
    print(item)
'''



'\nfor item in corpus_tfidf_trained:\n    print(item)\n'

In [7]:
model_trained.save(path+'cont_tfidf_model')

In [27]:
#change gensim's tfidf to sparse matrix
#corpus_tfidf_trained = models.load(path+'title_tfidf_model')
from scipy.sparse import csr_matrix
data = []
rows = []
cols = []
line_count = 0
for line in corpus_tfidf_trained:  # corpus_tfidf_trained 是之前由gensim生成的tfidf向量
    for elem in line:
        rows.append(line_count)
        cols.append(elem[0])
        data.append(elem[1])
    line_count += 1
tfidf_sparse_matrix = csr_matrix((data,(rows,cols))) # 稀疏向量
#tfidf_matrix = tfidf_sparse_matrix.toarray()  # 密集向量

In [28]:
#sparse matrix save
from scipy.io import mmwrite
mmwrite(path+'sp_cont_tfidf',tfidf_sparse_matrix)